In [1]:
# group by animal if can, otherwise session
# break up sessions
# dx vs. n

In [2]:
# check pvalues for STR neurons, if the format is like Hanks did, where he had it right in the data

In [3]:
using Revise, Distributed

addprocs(44);
@everywhere using Pkg; @everywhere Pkg.activate("/usr/people/briandd/Projects/pulse_input_DDM.jl")

In [18]:
@everywhere using pulse_input_DDM
using JLD2, PyPlot, Statistics

┌ Info: Precompiling pulse_input_DDM [c2542da2-1f7d-11e9-2506-87691095ebb2]
└ @ Base loading.jl:1192


      From worker 32:	WARNING: Method definition _bcs1(Any, Any) in module Broadcast at broadcast.jl:439 overwritten in module ImageFiltering at /usr/people/briandd/.julia/packages/ImageFiltering/8fmJ4/src/ImageFiltering.jl:28.
      From worker 42:	WARNING: Method definition _bcs1(Any, Any) in module Broadcast at broadcast.jl:439 overwritten in module ImageFiltering at /usr/people/briandd/.julia/packages/ImageFiltering/8fmJ4/src/ImageFiltering.jl:28.
      From worker 30:	WARNING: Method definition _bcs1(Any, Any) in module Broadcast at broadcast.jl:439 overwritten in module ImageFiltering at /usr/people/briandd/.julia/packages/ImageFiltering/8fmJ4/src/ImageFiltering.jl:28.
      From worker 23:	WARNING: Method definition _bcs1(Any, Any) in module Broadcast at broadcast.jl:439 overwritten in module ImageFiltering at /usr/people/briandd/.julia/packages/ImageFiltering/8fmJ4/src/ImageFiltering.jl:28.
      From worker 21:	WARNING: Method definition _bcs1(Any, Any) in module Broadcast at 

      From worker 16:	WARNING: Method definition _bcs1(Any, Any) in module Broadcast at broadcast.jl:439 overwritten in module ImageFiltering at /usr/people/briandd/.julia/packages/ImageFiltering/8fmJ4/src/ImageFiltering.jl:28.
      From worker 11:	WARNING: Method definition _bcs1(Any, Any) in module Broadcast at broadcast.jl:439 overwritten in module ImageFiltering at /usr/people/briandd/.julia/packages/ImageFiltering/8fmJ4/src/ImageFiltering.jl:28.
      From worker 28:	WARNING: Method definition _bcs1(Any, Any) in module Broadcast at broadcast.jl:439 overwritten in module ImageFiltering at /usr/people/briandd/.julia/packages/ImageFiltering/8fmJ4/src/ImageFiltering.jl:28.
      From worker 36:	WARNING: Method definition _bcs1(Any, Any) in module Broadcast at broadcast.jl:439 overwritten in module ImageFiltering at /usr/people/briandd/.julia/packages/ImageFiltering/8fmJ4/src/ImageFiltering.jl:28.
      From worker 17:	WARNING: Method definition _bcs1(Any, Any) in module Broadcast at 

In [19]:
path = ENV["HOME"]*"/Projects/pulse_input_DDM.jl"
f_str, n, dt = "sig", 103, 1e-2;

## individual rats

In [20]:
region = "FOF"

sessids, ratnames = sessids_from_region(region);

if (region == "STR") || (region == "FOF")
    delay = 0.05
elseif region == "PPC"
    delay = 0.1
end

0.05

In [27]:
rat = ratnames[1];
sesss = sessids[1][1:5];

In [28]:
data = aggregate_spiking_data(path*"/data/hanks_data_sessions/all_times", [sesss], [rat]);
use_bin_center = true;
data = map(x->bin_clicks_spikes_and_λ0!(x, use_bin_center; dt=dt,delay=delay), data);

In [ ]:
@time pz, py, = load_and_optimize(data, f_str, n; iterations=200, show_trace=true)

In [ ]:
JLD2.@save path*"/data/results/working/"*region*"_"*rat*".jld" pz py

In [ ]:
μ_hat_ct = pulse_input_DDM.sample_average_expected_rates_multiple_sessions(pz["final"], 
    py["final"], data, f_str);

In [ ]:
num_rows, num_cols = length(data), maximum(map(x-> x["N"], data))
fig, ax = subplots(num_rows, num_cols, figsize=(4*maximum(map(x-> x["N"], data)),4*length(data)))
my_colors = ["#E50000","#9F3F00","#5A7F00","#15BF00"]
#PPC colors = ["#1822A0","#5D4A7A","#A37354","#E99C2F"]
#STR colors = ["#A01892","#B85C71","#D0A150","#E9E62F"]

#fix k vs. n here, they are now mixed up

for i in 1:num_rows
    
    plt_data = data[i]
    plt_data_2 = μ_hat_ct[i]
    
    for j in 1:length(plt_data_2)               
        for k = 1:plt_data["nconds"]        
            
            if num_cols > 1
            
                ax[i,j].fill_between((1:length(plt_data["μ_ct"][j][k]))*dt,
                    plt_data["μ_ct"][j][k] + plt_data["σ_ct"][j][k],
                    plt_data["μ_ct"][j][k] - plt_data["σ_ct"][j][k],
                    alpha=0.2, color=my_colors[k])
                
                ax[i,j].plot((1:length(plt_data_2[j][k]))*dt,
                    plt_data_2[j][k], color=my_colors[k])                                     

                ax[i,j].set_xlim((0, 0.5))
                #ax[j].set_ylim((0, 30))

                
            else
                
                ax.fill_between((1:length(plt_data["μ_ct"][j][k]))*dt,
                    plt_data["μ_ct"][j][k] + plt_data["σ_ct"][j][k],
                    plt_data["μ_ct"][j][k] - plt_data["σ_ct"][j][k],
                    alpha=0.2, color=my_colors[k])
                
                ax.plot((1:length(plt_data_2[j][k]))*dt,
                    plt_data_2[j][k], color=my_colors[k])
            
                ax.set_xlim((0, 0.5))
                ax.set_ylim((0, 30))
                
            end
                        
        end   
        
        if num_cols > 1
            ax[i,j].plot((1:length(plt_data["μ_t"][j]))*dt,
                plt_data["μ_t"][j], color="black")
        else
            ax.plot((1:length(plt_data["μ_t"][j]))*dt,
                plt_data["μ_t"][j], color="black")
        end
                      
    end
    
end

tight_layout() 